In [ ]:
# =============================================================================
# CELL 1: Imports and Setup
# =============================================================================

import sys
import os
import json
import time
import shutil
import subprocess
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, Any, List, Optional

import pandas as pd
import numpy as np
import requests
import joblib

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Add project root to path
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

# Display header
print("=" * 60)
print("WEEK 7: MODEL DEPLOYMENT WITH FASTAPI")
print("=" * 60)
print(f"Project Root: {PROJECT_ROOT}")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)

WEEK 7: MODEL DEPLOYMENT WITH FASTAPI
Project Root: c:\Users\samue\Desktop\NSP\healthcare-appointments
Timestamp: 2025-12-04 02:46:30


In [ ]:
# =============================================================================
# CELL 2: Check Dependencies
# =============================================================================

print("\n📦 Checking Dependencies...")
print("-" * 40)

dependencies = {
    'fastapi': 'FastAPI',
    'uvicorn': 'Uvicorn',
    'pydantic': 'Pydantic',
    'httpx': 'HTTPX',
    'requests': 'Requests',
    'joblib': 'Joblib',
    'sklearn': 'Scikit-learn',
}

missing_deps = []

for module, name in dependencies.items():
    try:
        imported = __import__(module)
        version = getattr(imported, '__version__', 'installed')
        print(f"  ✅ {name}: {version}")
    except ImportError:
        print(f"  ❌ {name}: NOT INSTALLED")
        missing_deps.append(module)

if missing_deps:
    print(f"\n⚠️  Missing: {', '.join(missing_deps)}")
    print(f"   Run: pip install {' '.join(missing_deps)}")
else:
    print("\n✅ All dependencies installed!")


📦 Checking Dependencies...
----------------------------------------
  ✅ FastAPI: 0.119.0
  ✅ Uvicorn: 0.37.0
  ✅ Pydantic: 2.12.2
  ✅ HTTPX: 0.28.1
  ✅ Requests: 2.32.5
  ✅ Joblib: 1.5.2
  ✅ Scikit-learn: 1.7.2

✅ All dependencies installed!


In [ ]:
# =============================================================================
# CELL 3: Define Model Paths
# =============================================================================

# Define directory paths
MODELS_DIR = PROJECT_ROOT / "models"
TUNED_MODELS_DIR = MODELS_DIR / "tuned"
BASELINE_MODELS_DIR = MODELS_DIR / "baseline"
PRODUCTION_DIR = MODELS_DIR / "production"

# Create production directory if it doesn't exist
PRODUCTION_DIR.mkdir(parents=True, exist_ok=True)

print("📁 Model Directories:")
print(f"   Tuned: {TUNED_MODELS_DIR}")
print(f"   Baseline: {BASELINE_MODELS_DIR}")
print(f"   Production: {PRODUCTION_DIR}")

📁 Model Directories:
   Tuned: c:\Users\samue\Desktop\NSP\healthcare-appointments\models\tuned
   Baseline: c:\Users\samue\Desktop\NSP\healthcare-appointments\models\baseline
   Production: c:\Users\samue\Desktop\NSP\healthcare-appointments\models\production


In [ ]:
# =============================================================================
# CELL 4: Find Available Models
# =============================================================================

def find_models(directory: Path, pattern: str = "*.joblib") -> List[Path]:
    """Find all model files in a directory."""
    if directory.exists():
        return list(directory.glob(pattern))
    return []

# Find models
tuned_models = find_models(TUNED_MODELS_DIR, "*_tuned.joblib")
baseline_models = find_models(BASELINE_MODELS_DIR, "*.joblib")

print("\n🔍 Available Models:")
print("-" * 40)

print(f"\nTuned models ({len(tuned_models)}):")
for m in tuned_models:
    size_kb = m.stat().st_size / 1024
    print(f"   • {m.name} ({size_kb:.1f} KB)")

print(f"\nBaseline models ({len(baseline_models)}):")
for m in baseline_models:
    if 'preprocessor' not in m.name.lower():
        size_kb = m.stat().st_size / 1024
        print(f"   • {m.name} ({size_kb:.1f} KB)")


🔍 Available Models:
----------------------------------------

Tuned models (1):
   • random_forest_tuned.joblib (162101.9 KB)

Baseline models (6):
   • decision_tree.joblib (6.2 KB)
   • gradient_boosting.joblib (426.7 KB)
   • logistic_regression.joblib (1.2 KB)
   • random_forest.joblib (8836.6 KB)
   • xgboost.joblib (253.6 KB)


In [ ]:
# =============================================================================
# CELL 5: Select Best Model for Production
# =============================================================================

# Model priority order
MODEL_PRIORITY = ['xgboost', 'random_forest', 'gradient_boosting', 'logistic_regression']

selected_model_path = None
selected_model_name = None

# Check tuned models first
for model_name in MODEL_PRIORITY:
    for model_path in tuned_models:
        if model_name in model_path.name.lower():
            selected_model_path = model_path
            selected_model_name = model_name
            break
    if selected_model_path:
        break

# Fall back to baseline models
if not selected_model_path:
    for model_name in MODEL_PRIORITY:
        for model_path in baseline_models:
            if model_name in model_path.name.lower() and 'preprocessor' not in model_path.name.lower():
                selected_model_path = model_path
                selected_model_name = model_name
                break
        if selected_model_path:
            break

# Display result
if selected_model_path:
    print(f"\n✅ Selected Model: {selected_model_name}")
    print(f"   Path: {selected_model_path}")
else:
    print("\n⚠️ No trained model found - will create demo model")


✅ Selected Model: random_forest
   Path: c:\Users\samue\Desktop\NSP\healthcare-appointments\models\tuned\random_forest_tuned.joblib


In [ ]:
# =============================================================================
# CELL 6: Prepare Production Model
# =============================================================================

model = None

if selected_model_path and selected_model_path.exists():
    # Load existing model
    print(f"\n📦 Loading model from: {selected_model_path}")
    model = joblib.load(selected_model_path)
    print(f"   Type: {type(model).__name__}")
    
    # Copy to production directory
    production_model_path = PRODUCTION_DIR / "model.joblib"
    shutil.copy(selected_model_path, production_model_path)
    print(f"   ✅ Copied to: {production_model_path}")
    
else:
    # Create demo model
    print("\n📦 Creating demonstration model...")
    from sklearn.ensemble import RandomForestClassifier
    
    np.random.seed(42)
    X_dummy = np.random.rand(1000, 15)
    y_dummy = np.random.randint(0, 2, 1000)
    
    model = RandomForestClassifier(
        n_estimators=50,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_dummy, y_dummy)
    
    selected_model_name = "random_forest_demo"
    
    # Save demo model
    production_model_path = PRODUCTION_DIR / "model.joblib"
    joblib.dump(model, production_model_path)
    print(f"   ✅ Demo model saved to: {production_model_path}")


📦 Loading model from: c:\Users\samue\Desktop\NSP\healthcare-appointments\models\tuned\random_forest_tuned.joblib
   Type: Pipeline
   ✅ Copied to: c:\Users\samue\Desktop\NSP\healthcare-appointments\models\production\model.joblib


In [ ]:
# =============================================================================
# CELL 7: Copy Preprocessor
# =============================================================================

preprocessor_sources = [
    TUNED_MODELS_DIR / "preprocessor.joblib",
    BASELINE_MODELS_DIR / "preprocessor.joblib",
]

preprocessor_copied = False

for preprocessor_path in preprocessor_sources:
    if preprocessor_path.exists():
        production_preprocessor_path = PRODUCTION_DIR / "preprocessor.joblib"
        shutil.copy(preprocessor_path, production_preprocessor_path)
        print(f"✅ Preprocessor copied from: {preprocessor_path}")
        preprocessor_copied = True
        break

if not preprocessor_copied:
    print("⚠️ No preprocessor found - API will use simplified processing")

✅ Preprocessor copied from: c:\Users\samue\Desktop\NSP\healthcare-appointments\models\baseline\preprocessor.joblib


In [ ]:
# =============================================================================
# CELL 8: Create Model Metadata
# =============================================================================

model_metadata = {
    "model_name": selected_model_name,
    "model_version": "1.0.0",
    "model_type": type(model).__name__ if model else "Unknown",
    "created_at": datetime.now().isoformat(),
    "created_by": "Week 7 Deployment Notebook",
    "source_path": str(selected_model_path) if selected_model_path else "generated",
    "description": "Production model for healthcare no-show prediction",
    "input_features": {
        "required": ["age", "gender", "lead_days"],
        "optional": [
            "scholarship", "hypertension", "diabetes",
            "alcoholism", "handicap", "sms_received",
            "neighbourhood", "appointment_weekday"
        ]
    },
    "output": {
        "prediction": "Binary (0=show, 1=no-show)",
        "probability": "Float (0-1)",
        "risk_tier": "MINIMAL/LOW/MEDIUM/HIGH/CRITICAL"
    }
}

# Add model-specific attributes
if model and hasattr(model, 'n_estimators'):
    model_metadata['n_estimators'] = model.n_estimators
if model and hasattr(model, 'max_depth'):
    model_metadata['max_depth'] = model.max_depth
if model and hasattr(model, 'feature_importances_'):
    model_metadata['n_features'] = len(model.feature_importances_)

# Save metadata
metadata_path = PRODUCTION_DIR / "model_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(model_metadata, f, indent=2)

print(f"✅ Metadata saved to: {metadata_path}")
print("\n📋 Model Metadata:")
print(json.dumps(model_metadata, indent=2))

✅ Metadata saved to: c:\Users\samue\Desktop\NSP\healthcare-appointments\models\production\model_metadata.json

📋 Model Metadata:
{
  "model_name": "random_forest",
  "model_version": "1.0.0",
  "model_type": "Pipeline",
  "created_at": "2025-12-04T02:46:32.483748",
  "created_by": "Week 7 Deployment Notebook",
  "source_path": "c:\\Users\\samue\\Desktop\\NSP\\healthcare-appointments\\models\\tuned\\random_forest_tuned.joblib",
  "description": "Production model for healthcare no-show prediction",
  "input_features": {
    "required": [
      "age",
      "gender",
      "lead_days"
    ],
    "optional": [
      "scholarship",
      "hypertension",
      "diabetes",
      "alcoholism",
      "handicap",
      "sms_received",
      "neighbourhood",
      "appointment_weekday"
    ]
  },
  "output": {
    "prediction": "Binary (0=show, 1=no-show)",
    "probability": "Float (0-1)",
    "risk_tier": "MINIMAL/LOW/MEDIUM/HIGH/CRITICAL"
  }
}


In [ ]:
# =============================================================================
# CELL 9: Verify Production Directory
# =============================================================================

print("\n" + "=" * 60)
print("PRODUCTION DIRECTORY CONTENTS")
print("=" * 60)

total_size = 0
for f in sorted(PRODUCTION_DIR.iterdir()):
    size_kb = f.stat().st_size / 1024
    total_size += size_kb
    print(f"   {f.name}: {size_kb:.1f} KB")

print(f"\n   Total size: {total_size:.1f} KB")
print("=" * 60)


PRODUCTION DIRECTORY CONTENTS
   feature_importance.json: 2.2 KB
   model.joblib: 162101.9 KB
   model_metadata.json: 0.8 KB
   preprocessor.joblib: 13.0 KB
   shap_explainer.joblib: 0.0 KB

   Total size: 162118.0 KB


In [ ]:
# =============================================================================
# CELL 10: API Configuration
# =============================================================================

# API Settings
API_HOST = "127.0.0.1"
API_PORT = 8000
API_URL = f"http://{API_HOST}:{API_PORT}"

print("=" * 60)
print("API SERVER CONFIGURATION")
print("=" * 60)
print(f"   Host: {API_HOST}")
print(f"   Port: {API_PORT}")
print(f"   Base URL: {API_URL}")
print(f"   Docs URL: {API_URL}/docs")
print(f"   Health URL: {API_URL}/health")
print("=" * 60)

API SERVER CONFIGURATION
   Host: 127.0.0.1
   Port: 8000
   Base URL: http://127.0.0.1:8000
   Docs URL: http://127.0.0.1:8000/docs
   Health URL: http://127.0.0.1:8000/health


In [ ]:
# =============================================================================
# CELL 11: API Status Check Functions
# =============================================================================

def check_api_status(url: str = API_URL, timeout: int = 2) -> Dict[str, Any]:
    """
    Check if the API is running and return status information.
    
    Parameters
    ----------
    url : str
        Base URL of the API
    timeout : int
        Request timeout in seconds
    
    Returns
    -------
    dict
        Status information including running, healthy, and data/error
    """
    try:
        response = requests.get(f"{url}/health", timeout=timeout)
        if response.status_code == 200:
            return {
                "running": True,
                "healthy": True,
                "data": response.json()
            }
        return {
            "running": True,
            "healthy": False,
            "status_code": response.status_code
        }
    except requests.ConnectionError:
        return {"running": False, "healthy": False, "error": "Connection refused"}
    except requests.Timeout:
        return {"running": False, "healthy": False, "error": "Timeout"}
    except Exception as e:
        return {"running": False, "healthy": False, "error": str(e)}


def wait_for_api(url: str = API_URL, max_wait: int = 30) -> bool:
    """
    Wait for API to become available.
    
    Parameters
    ----------
    url : str
        Base URL of the API
    max_wait : int
        Maximum seconds to wait
    
    Returns
    -------
    bool
        True if API is ready, False otherwise
    """
    print(f"Waiting for API at {url}...")
    for i in range(max_wait):
        status = check_api_status(url)
        if status["running"] and status["healthy"]:
            print(f"✅ API ready after {i} seconds")
            return True
        time.sleep(1)
        if i > 0 and i % 5 == 0:
            print(f"   Still waiting... ({i}s)")
    print(f"❌ API not ready after {max_wait} seconds")
    return False

In [ ]:
# =============================================================================
# CELL 12: Check Current API Status
# =============================================================================

status = check_api_status()

print("\n📡 Current API Status:")
print("-" * 40)

if status["running"]:
    print("   ✅ API is running")
    if status["healthy"]:
        print("   ✅ Health check passed")
        if "data" in status:
            data = status["data"]
            print(f"   Model loaded: {data.get('model_loaded', 'unknown')}")
            print(f"   Version: {data.get('version', 'unknown')}")
    else:
        print(f"   ⚠️ Health check failed (status: {status.get('status_code', 'unknown')})")
else:
    print("   ❌ API is NOT running")
    print(f"   Error: {status.get('error', 'unknown')}")
    print("\n" + "=" * 60)
    print("⚠️  PLEASE START THE API SERVER")
    print("=" * 60)
    print(f"""
Open a new terminal and run:

    cd {PROJECT_ROOT}
    python serve_model.py --reload

Then continue with the next cells.
""")


📡 Current API Status:
----------------------------------------
   ✅ API is running
   ✅ Health check passed
   Model loaded: unknown
   Version: unknown


In [ ]:
# =============================================================================
# CELL 13: Test Helper Decorator
# =============================================================================

def api_test(test_name: str):
    """
    Decorator to wrap API tests with consistent formatting and error handling.
    
    Parameters
    ----------
    test_name : str
        Name of the test for display
    """
    def decorator(func):
        def wrapper(*args, **kwargs):
            print(f"\n{'=' * 60}")
            print(f"TEST: {test_name}")
            print('=' * 60)
            
            if not check_api_status()["running"]:
                print("⚠️ API not running - skipping test")
                print("   Start server with: python serve_model.py")
                return None
            
            try:
                return func(*args, **kwargs)
            except requests.exceptions.ConnectionError:
                print("❌ Connection error - API may have stopped")
                return None
            except Exception as e:
                print(f"❌ Test failed: {type(e).__name__}: {e}")
                return None
        return wrapper
    return decorator

print("✅ Test helper decorator defined")

✅ Test helper decorator defined


In [ ]:
# =============================================================================
# CELL 14: Test Health Endpoint
# =============================================================================

@api_test("Health Check Endpoint")
def test_health_endpoint():
    """Test the /health endpoint."""
    
    response = requests.get(f"{API_URL}/health")
    print(f"Status Code: {response.status_code}")
    
    if response.status_code == 200:
        data = response.json()
        
        print(f"\n📊 Health Status:")
        print(f"   Status: {data.get('status', 'unknown')}")
        print(f"   Model Loaded: {data.get('model_loaded', 'unknown')}")
        print(f"   Version: {data.get('version', 'unknown')}")
        
        if data.get('model_info'):
            print(f"\n📦 Model Info:")
            model_info = data['model_info']
            print(f"   Name: {model_info.get('name', 'unknown')}")
            print(f"   Type: {model_info.get('type', 'unknown')}")
            print(f"   Features: {model_info.get('features', 'unknown')}")
        
        return data
    else:
        print(f"❌ Unexpected response: {response.text}")
        return None

# Run the test
health_result = test_health_endpoint()


TEST: Health Check Endpoint
Status Code: 200

📊 Health Status:
   Status: healthy
   Model Loaded: unknown
   Version: unknown


In [ ]:
# =============================================================================
# CELL 15: Test Kubernetes-Style Probes
# =============================================================================

@api_test("Readiness and Liveness Probes")
def test_probes():
    """Test /ready and /live endpoints for Kubernetes compatibility."""
    
    # Readiness probe
    print("1. Readiness Probe (/ready):")
    response = requests.get(f"{API_URL}/ready")
    print(f"   Status Code: {response.status_code}")
    if response.status_code == 200:
        data = response.json()
        print(f"   Ready: {data.get('ready', 'unknown')}")
        print(f"   Timestamp: {data.get('timestamp', 'unknown')}")
    
    # Liveness probe
    print("\n2. Liveness Probe (/live):")
    response = requests.get(f"{API_URL}/live")
    print(f"   Status Code: {response.status_code}")
    if response.status_code == 200:
        data = response.json()
        print(f"   Alive: {data.get('alive', 'unknown')}")
        print(f"   Timestamp: {data.get('timestamp', 'unknown')}")
    
    return True

test_probes()


TEST: Readiness and Liveness Probes
1. Readiness Probe (/ready):
   Status Code: 200
   Ready: True
   Timestamp: 2025-12-04T02:46:32.786937

2. Liveness Probe (/live):
   Status Code: 200
   Alive: True
   Timestamp: 2025-12-04T02:46:32.789468


True

In [ ]:
# =============================================================================
# CELL 16: Test Single Prediction Endpoint
# =============================================================================

@api_test("Single Prediction")
def test_single_prediction():
    """Test POST /api/v1/predict with a sample appointment."""
    
    # Sample appointment data
    appointment = {
        "age": 35,
        "gender": "F",
        "scholarship": 0,
        "hypertension": 0,
        "diabetes": 0,
        "alcoholism": 0,
        "handicap": 0,
        "sms_received": 1,
        "lead_days": 7
    }
    
    print("📝 Request Body:")
    print(json.dumps(appointment, indent=2))
    
    # Make request
    response = requests.post(
        f"{API_URL}/api/v1/predict",
        json=appointment,
        headers={"Content-Type": "application/json"}
    )
    
    print(f"\n📬 Response Status: {response.status_code}")
    
    if response.status_code == 200:
        result = response.json()
        
        # Display prediction
        prediction_text = "No-Show" if result['prediction'] == 1 else "Will Attend"
        print(f"\n📊 Prediction Result:")
        print(f"   Prediction: {prediction_text}")
        print(f"   Probability: {result['probability']:.1%}")
        
        # Display risk assessment
        risk = result.get('risk', {})
        print(f"\n🎯 Risk Assessment:")
        print(f"   Tier: {risk.get('tier', 'unknown')} {risk.get('emoji', '')}")
        print(f"   Confidence: {risk.get('confidence', 'unknown')}")
        print(f"   Color: {risk.get('color', 'unknown')}")
        
        # Display intervention
        intervention = result.get('intervention', {})
        print(f"\n💡 Recommended Intervention:")
        print(f"   Action: {intervention.get('action', 'unknown')}")
        print(f"   SMS Reminders: {intervention.get('sms_reminders', 'unknown')}")
        print(f"   Phone Call: {intervention.get('phone_call', 'unknown')}")
        print(f"   Priority: {intervention.get('priority', 'unknown')}")
        
        # Display metadata
        print(f"\n📋 Metadata:")
        print(f"   Model Version: {result.get('model_version', 'unknown')}")
        print(f"   Prediction ID: {result.get('prediction_id', 'unknown')}")
        print(f"   Timestamp: {result.get('timestamp', 'unknown')}")
        
        return result
    else:
        print(f"❌ Error Response: {response.text}")
        return None

# Run the test
prediction_result = test_single_prediction()


TEST: Single Prediction
📝 Request Body:
{
  "age": 35,
  "gender": "F",
  "scholarship": 0,
  "hypertension": 0,
  "diabetes": 0,
  "alcoholism": 0,
  "handicap": 0,
  "sms_received": 1,
  "lead_days": 7
}

📬 Response Status: 200

📊 Prediction Result:
   Prediction: Will Attend
   Probability: 27.7%

🎯 Risk Assessment:
   Tier: LOW 🟢
   Confidence: Moderate
   Color: #2ecc71

💡 Recommended Intervention:
   Action: Standard SMS reminder
   SMS Reminders: 1
   Phone Call: False
   Priority: normal

📋 Metadata:
   Model Version: 2.1.0
   Prediction ID: c64bfc50
   Timestamp: 2025-12-04T02:46:32.823314


In [ ]:
# =============================================================================
# CELL 17: Test Multiple Risk Scenarios
# =============================================================================

@api_test("Multiple Risk Scenarios")
def test_risk_scenarios():
    """Test predictions across different risk profiles."""
    
    scenarios = [
        {
            "name": "Low Risk - Senior with chronic condition",
            "data": {
                "age": 68,
                "gender": "F",
                "lead_days": 2,
                "hypertension": 1,
                "diabetes": 1,
                "sms_received": 1
            }
        },
        {
            "name": "Medium Risk - Adult, moderate lead time",
            "data": {
                "age": 42,
                "gender": "M",
                "lead_days": 12,
                "sms_received": 1
            }
        },
        {
            "name": "High Risk - Young adult, long lead time, no SMS",
            "data": {
                "age": 23,
                "gender": "M",
                "lead_days": 28,
                "sms_received": 0,
                "scholarship": 1
            }
        },
        {
            "name": "First-time patient",
            "data": {
                "age": 30,
                "gender": "F",
                "lead_days": 14,
                "sms_received": 1
            }
        }
    ]
    
    results = []
    
    for i, scenario in enumerate(scenarios, 1):
        print(f"\n{'─' * 50}")
        print(f"Scenario {i}: {scenario['name']}")
        print(f"{'─' * 50}")
        
        response = requests.post(
            f"{API_URL}/api/v1/predict",
            json=scenario["data"]
        )
        
        if response.status_code == 200:
            result = response.json()
            
            print(f"   Input: age={scenario['data']['age']}, "
                  f"lead_days={scenario['data']['lead_days']}, "
                  f"sms={scenario['data'].get('sms_received', 0)}")
            print(f"   → Probability: {result['probability']:.1%}")
            print(f"   → Risk Tier: {result['risk']['tier']} {result['risk']['emoji']}")
            print(f"   → Action: {result['intervention']['action']}")
            
            results.append({
                "scenario": scenario["name"],
                "probability": result['probability'],
                "risk_tier": result['risk']['tier']
            })
        else:
            print(f"   ❌ Error: {response.status_code}")
    
    return results

# Run the test
scenario_results = test_risk_scenarios()


TEST: Multiple Risk Scenarios

──────────────────────────────────────────────────
Scenario 1: Low Risk - Senior with chronic condition
──────────────────────────────────────────────────
   Input: age=68, lead_days=2, sms=1
   → Probability: 19.7%
   → Risk Tier: LOW 🟢
   → Action: Standard SMS reminder

──────────────────────────────────────────────────
Scenario 2: Medium Risk - Adult, moderate lead time
──────────────────────────────────────────────────
   Input: age=42, lead_days=12, sms=1
   → Probability: 29.6%
   → Risk Tier: LOW 🟢
   → Action: Standard SMS reminder

──────────────────────────────────────────────────
Scenario 3: High Risk - Young adult, long lead time, no SMS
──────────────────────────────────────────────────
   Input: age=23, lead_days=28, sms=0
   → Probability: 45.0%
   → Risk Tier: MEDIUM 🟡
   → Action: Double SMS reminder

──────────────────────────────────────────────────
Scenario 4: First-time patient
──────────────────────────────────────────────────
   I

In [ ]:
# =============================================================================
# CELL 18: Test Batch Prediction Endpoint
# =============================================================================

@api_test("Batch Prediction")
def test_batch_prediction():
    """Test POST /api/v1/predict/batch with multiple appointments."""
    
    # Create diverse batch
    batch_appointments = [
        {"age": 22, "gender": "M", "lead_days": 1, "sms_received": 1},
        {"age": 28, "gender": "F", "lead_days": 5, "sms_received": 1},
        {"age": 35, "gender": "M", "lead_days": 10, "sms_received": 0},
        {"age": 42, "gender": "F", "lead_days": 14, "sms_received": 1, "hypertension": 1},
        {"age": 55, "gender": "M", "lead_days": 21, "sms_received": 1, "diabetes": 1},
        {"age": 65, "gender": "F", "lead_days": 7, "sms_received": 1, "hypertension": 1},
        {"age": 75, "gender": "M", "lead_days": 3, "sms_received": 1},
        {"age": 18, "gender": "F", "lead_days": 30, "sms_received": 0, "scholarship": 1},
    ]
    
    print(f"📦 Sending batch of {len(batch_appointments)} appointments...")
    
    response = requests.post(
        f"{API_URL}/api/v1/predict/batch",
        json={"appointments": batch_appointments}
    )
    
    print(f"Status Code: {response.status_code}")
    
    if response.status_code == 200:
        result = response.json()
        summary = result["summary"]
        
        print(f"\n📊 Batch Results Summary:")
        print(f"   Total Appointments: {summary['total']}")
        print(f"   Predicted No-Shows: {summary['predicted_noshows']}")
        print(f"   Predicted Shows: {summary['predicted_shows']}")
        print(f"   Average Probability: {summary['avg_probability']:.1%}")
        print(f"   Processing Time: {result['processing_time_ms']:.1f}ms")
        
        print(f"\n📈 Risk Distribution:")
        risk_dist = summary['risk_distribution']
        total = summary['total']
        
        for tier in ['CRITICAL', 'HIGH', 'MEDIUM', 'LOW', 'MINIMAL']:
            count = risk_dist.get(tier, 0)
            pct = count / total * 100 if total > 0 else 0
            bar = '█' * int(pct / 5)
            print(f"   {tier:10} │ {bar:20} │ {count} ({pct:.1f}%)")
        
        return result
    else:
        print(f"❌ Error: {response.text}")
        return None

# Run the test
batch_result = test_batch_prediction()


TEST: Batch Prediction
📦 Sending batch of 8 appointments...


Status Code: 200

📊 Batch Results Summary:
   Total Appointments: 8
   Predicted No-Shows: 0
   Predicted Shows: 8
   Average Probability: 28.3%
   Processing Time: 118.4ms

📈 Risk Distribution:
   CRITICAL   │                      │ 0 (0.0%)
   HIGH       │                      │ 0 (0.0%)
   MEDIUM     │ █████                │ 2 (25.0%)
   LOW        │ ███████████████      │ 6 (75.0%)
   MINIMAL    │                      │ 0 (0.0%)


In [ ]:
# =============================================================================
# CELL 19: Test Input Validation
# =============================================================================

@api_test("Input Validation")
def test_input_validation():
    """Test that the API properly validates input data."""
    
    invalid_cases = [
        {
            "name": "Negative age",
            "data": {"age": -5, "gender": "F", "lead_days": 7},
            "expected_status": 422
        },
        {
            "name": "Age too high (>120)",
            "data": {"age": 150, "gender": "F", "lead_days": 7},
            "expected_status": 422
        },
        {
            "name": "Invalid gender",
            "data": {"age": 35, "gender": "X", "lead_days": 7},
            "expected_status": 422
        },
        {
            "name": "Negative lead days",
            "data": {"age": 35, "gender": "F", "lead_days": -1},
            "expected_status": 422
        },
        {
            "name": "Missing required field (lead_days)",
            "data": {"age": 35, "gender": "F"},
            "expected_status": 422
        },
        {
            "name": "Invalid binary field (sms_received=5)",
            "data": {"age": 35, "gender": "F", "lead_days": 7, "sms_received": 5},
            "expected_status": 422
        }
    ]
    
    results = []
    passed = 0
    failed = 0
    
    for case in invalid_cases:
        response = requests.post(
            f"{API_URL}/api/v1/predict",
            json=case["data"]
        )
        
        is_passed = response.status_code == case["expected_status"]
        icon = "✅" if is_passed else "❌"
        
        print(f"{icon} {case['name']}")
        print(f"   Expected: {case['expected_status']}, Got: {response.status_code}")
        
        if is_passed:
            passed += 1
        else:
            failed += 1
        
        results.append({
            "case": case["name"],
            "passed": is_passed,
            "expected": case["expected_status"],
            "actual": response.status_code
        })
    
    print(f"\n📊 Validation Tests: {passed}/{len(invalid_cases)} passed")
    
    return results

# Run the test
validation_results = test_input_validation()


TEST: Input Validation
✅ Negative age
   Expected: 422, Got: 422
✅ Age too high (>120)
   Expected: 422, Got: 422
✅ Invalid gender
   Expected: 422, Got: 422
✅ Negative lead days
   Expected: 422, Got: 422
✅ Missing required field (lead_days)
   Expected: 422, Got: 422
✅ Invalid binary field (sms_received=5)
   Expected: 422, Got: 422

📊 Validation Tests: 6/6 passed


In [ ]:
# =============================================================================
# CELL 20: Test Quick Prediction Endpoint
# =============================================================================

@api_test("Quick Prediction (Query Parameters)")
def test_quick_prediction():
    """Test POST /api/v1/predict/quick using query parameters."""
    
    params = {
        "age": 35,
        "gender": "F",
        "lead_days": 7,
        "sms_received": 1
    }
    
    print("📝 Query Parameters:")
    for key, value in params.items():
        print(f"   {key}={value}")
    
    response = requests.post(
        f"{API_URL}/api/v1/predict/quick",
        params=params
    )
    
    print(f"\nStatus Code: {response.status_code}")
    
    if response.status_code == 200:
        result = response.json()
        
        print(f"\n📊 Quick Prediction Result:")
        print(f"   Probability: {result.get('probability', 0):.1%}")
        print(f"   Prediction: {result.get('prediction', 'unknown')}")
        print(f"   Risk Tier: {result.get('risk_tier', 'unknown')}")
        print(f"   Intervention: {result.get('intervention', 'unknown')}")
        
        return result
    else:
        print(f"❌ Error: {response.text}")
        return None

# Run the test
quick_result = test_quick_prediction()


TEST: Quick Prediction (Query Parameters)
📝 Query Parameters:
   age=35
   gender=F
   lead_days=7
   sms_received=1



Status Code: 200

📊 Quick Prediction Result:
   Probability: 27.7%
   Prediction: Will Attend
   Risk Tier: LOW
   Intervention: Standard SMS reminder


In [ ]:
# =============================================================================
# CELL 21: Test Model Information Endpoints
# =============================================================================

@api_test("Model Information Endpoints")
def test_model_info():
    """Test model information and feature endpoints."""
    
    # 1. Model info
    print("1. Model Information (/api/v1/model):")
    print("-" * 40)
    response = requests.get(f"{API_URL}/api/v1/model")
    if response.status_code == 200:
        data = response.json()
        print(f"   Name: {data.get('name', 'unknown')}")
        print(f"   Version: {data.get('version', 'unknown')}")
        print(f"   Type: {data.get('type', 'unknown')}")
        print(f"   Features: {data.get('features', 'unknown')}")
    else:
        print(f"   ❌ Error: {response.status_code}")
    
    # 2. Features info
    print("\n2. Feature Information (/api/v1/model/features):")
    print("-" * 40)
    response = requests.get(f"{API_URL}/api/v1/model/features")
    if response.status_code == 200:
        data = response.json()
        print(f"   Required: {data.get('required', [])}")
        optional = data.get('optional', [])
        print(f"   Optional: {len(optional)} features")
        if optional:
            print(f"            {optional[:5]}...")
    else:
        print(f"   ❌ Error: {response.status_code}")
    
    # 3. Threshold info
    print("\n3. Threshold Information (/api/v1/predict/thresholds):")
    print("-" * 40)
    response = requests.get(f"{API_URL}/api/v1/predict/thresholds")
    if response.status_code == 200:
        data = response.json()
        print(f"   Default Threshold: {data.get('default_threshold', 'unknown')}")
        tiers = data.get('risk_tiers', {})
        print(f"   Risk Tiers Defined: {len(tiers)}")
        for tier_name, tier_info in list(tiers.items())[:3]:
            print(f"      • {tier_name}: min_prob={tier_info.get('min_probability', 'unknown')}")
    else:
        print(f"   ❌ Error: {response.status_code}")
    
    return True

# Run the test
test_model_info()


TEST: Model Information Endpoints
1. Model Information (/api/v1/model):
----------------------------------------
   Name: unknown
   Version: unknown
   Type: unknown
   Features: unknown

2. Feature Information (/api/v1/model/features):
----------------------------------------
   Required: ['age', 'gender', 'lead_days']
   Optional: 10 features
            ['handicap', 'scholarship', 'alcoholism', 'is_first_appointment', 'diabetes']...

3. Threshold Information (/api/v1/predict/thresholds):
----------------------------------------
   Default Threshold: 0.5
   Risk Tiers Defined: 5
      • CRITICAL: min_prob=0.7
      • HIGH: min_prob=0.5
      • MEDIUM: min_prob=0.3


True

In [ ]:
# =============================================================================
# CELL 22: Performance Benchmarks
# =============================================================================

import statistics
import concurrent.futures

@api_test("Performance Benchmarks")
def test_performance():
    """Measure API performance metrics."""
    
    sample_appointment = {
        "age": 35,
        "gender": "F",
        "lead_days": 7,
        "sms_received": 1
    }
    
    # 1. Single request latency
    print("1. Single Request Latency (20 requests):")
    print("-" * 40)
    
    latencies = []
    for i in range(20):
        start = time.time()
        response = requests.post(
            f"{API_URL}/api/v1/predict",
            json=sample_appointment
        )
        elapsed = (time.time() - start) * 1000  # Convert to ms
        if response.status_code == 200:
            latencies.append(elapsed)
    
    if latencies:
        print(f"   Mean:    {statistics.mean(latencies):.1f}ms")
        print(f"   Median:  {statistics.median(latencies):.1f}ms")
        print(f"   Std Dev: {statistics.stdev(latencies):.1f}ms")
        print(f"   Min:     {min(latencies):.1f}ms")
        print(f"   Max:     {max(latencies):.1f}ms")
    
    # 2. Batch request performance
    print("\n2. Batch Request Performance:")
    print("-" * 40)
    
    batch_sizes = [10, 50, 100]
    for size in batch_sizes:
        appointments = [sample_appointment.copy() for _ in range(size)]
        
        start = time.time()
        response = requests.post(
            f"{API_URL}/api/v1/predict/batch",
            json={"appointments": appointments}
        )
        elapsed = (time.time() - start) * 1000
        
        if response.status_code == 200:
            per_item = elapsed / size
            print(f"   {size:3} items: {elapsed:6.1f}ms total, {per_item:.2f}ms/item")
    
    # 3. Concurrent requests
    print("\n3. Concurrent Requests (10 simultaneous):")
    print("-" * 40)
    
    def make_single_request():
        start = time.time()
        response = requests.post(
            f"{API_URL}/api/v1/predict",
            json=sample_appointment
        )
        return (time.time() - start) * 1000, response.status_code
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(make_single_request) for _ in range(10)]
        results = [f.result() for f in futures]
    
    times = [r[0] for r in results]
    success_count = sum(1 for r in results if r[1] == 200)
    
    print(f"   Successful: {success_count}/10")
    print(f"   Mean Latency: {statistics.mean(times):.1f}ms")
    print(f"   Max Latency: {max(times):.1f}ms")
    
    # Estimate throughput
    if times:
        throughput = len(times) / (max(times) / 1000)
        print(f"   Est. Throughput: {throughput:.0f} req/sec")
    
    return {
        "single_latency_ms": statistics.mean(latencies) if latencies else None,
        "concurrent_success": success_count
    }

# Run the test
performance_results = test_performance()


TEST: Performance Benchmarks
1. Single Request Latency (20 requests):
----------------------------------------
   Mean:    17.7ms
   Median:  17.8ms
   Std Dev: 1.6ms
   Min:     15.1ms
   Max:     20.0ms

2. Batch Request Performance:
----------------------------------------
    10 items:  227.8ms total, 22.78ms/item
    50 items:  694.1ms total, 13.88ms/item
   100 items: 1437.0ms total, 14.37ms/item

3. Concurrent Requests (10 simultaneous):
----------------------------------------
   Successful: 10/10
   Mean Latency: 132.6ms
   Max Latency: 161.2ms
   Est. Throughput: 62 req/sec


In [ ]:
# =============================================================================
# CELL 23: Python API Client Demonstration
# =============================================================================

print("=" * 60)
print("PYTHON CLIENT LIBRARY DEMO")
print("=" * 60)

# Try to import and use the client
try:
    # Add scripts directory to path
    scripts_path = PROJECT_ROOT / "scripts"
    if scripts_path.exists():
        sys.path.insert(0, str(scripts_path))
    
    from api_client import NoShowAPIClient, PredictionResult
    
    print("✅ API Client imported successfully\n")
    
    # Check if API is running
    if check_api_status()["running"]:
        # Create client instance
        client = NoShowAPIClient(API_URL)
        
        # Health check
        print("📋 Client Health Check:")
        if client.is_healthy():
            print("   ✅ API is healthy\n")
        else:
            print("   ⚠️ API health check failed\n")
        
        # Make a prediction using the client
        print("📊 Making Prediction with Client:")
        print("-" * 40)
        
        result = client.predict(
            age=35,
            gender="F",
            lead_days=7,
            sms_received=1,
            hypertension=0,
            diabetes=0
        )
        
        # Display result using the PredictionResult object
        print(f"\n{result}")
        
        # Access specific properties
        print(f"\n📋 Additional Properties:")
        print(f"   result.will_show: {result.will_show}")
        print(f"   result.will_noshow: {result.will_noshow}")
        print(f"   result.probability: {result.probability:.4f}")
        print(f"   result.risk_tier: {result.risk_tier}")
        
    else:
        print("⚠️ API not running - cannot demonstrate client")
        print("   Start server with: python serve_model.py")

except ImportError as e:
    print(f"⚠️ Could not import API client: {e}")
    print("   Ensure scripts/api_client.py exists")
except Exception as e:
    print(f"❌ Error: {type(e).__name__}: {e}")

PYTHON CLIENT LIBRARY DEMO
✅ API Client imported successfully

📋 Client Health Check:
   ✅ API is healthy

📊 Making Prediction with Client:
----------------------------------------



Prediction: WILL ATTEND
Probability: 27.7%
Risk Tier: LOW
Intervention: Standard SMS reminder

📋 Additional Properties:
   result.will_show: True
   result.will_noshow: False
   result.probability: 0.2771
   result.risk_tier: LOW


In [ ]:
# =============================================================================
# CELL 24: Batch Prediction with Client
# =============================================================================

if check_api_status()["running"]:
    try:
        print("=" * 60)
        print("BATCH PREDICTION WITH CLIENT")
        print("=" * 60)
        
        # Create list of appointments
        appointments = [
            {"age": 25, "gender": "M", "lead_days": 3, "sms_received": 1},
            {"age": 40, "gender": "F", "lead_days": 10, "sms_received": 1},
            {"age": 55, "gender": "M", "lead_days": 20, "sms_received": 0},
            {"age": 70, "gender": "F", "lead_days": 5, "sms_received": 1, "hypertension": 1},
        ]
        
        print(f"\n📦 Sending {len(appointments)} appointments...")
        
        # Use client for batch prediction
        result = client.predict_batch(appointments)
        
        print(f"\n📊 Batch Results:")
        print(f"   Total: {result['summary']['total']}")
        print(f"   Predicted No-Shows: {result['summary']['predicted_noshows']}")
        print(f"   Predicted Shows: {result['summary']['predicted_shows']}")
        print(f"   Average Probability: {result['summary']['avg_probability']:.1%}")
        print(f"   Processing Time: {result['processing_time_ms']:.1f}ms")
        
    except NameError:
        print("⚠️ Client not defined - run the previous cell first")
    except Exception as e:
        print(f"❌ Error: {type(e).__name__}: {e}")
else:
    print("⚠️ API not running - skipping batch demo")

BATCH PREDICTION WITH CLIENT

📦 Sending 4 appointments...

📊 Batch Results:
   Total: 4
   Predicted No-Shows: 0
   Predicted Shows: 4
   Average Probability: 26.8%
   Processing Time: 61.5ms


In [ ]:
# =============================================================================
# CELL 25: Docker Status Check
# =============================================================================

print("=" * 60)
print("DOCKER DEPLOYMENT STATUS")
print("=" * 60)

# Check if Docker is available
docker_path = shutil.which("docker")
docker_available = docker_path is not None

if docker_available:
    print("✅ Docker is installed")
    
    # Get Docker version
    try:
        result = subprocess.run(
            ["docker", "--version"],
            capture_output=True,
            text=True,
            timeout=5
        )
        print(f"   {result.stdout.strip()}")
    except Exception as e:
        print(f"   Could not get version: {e}")
    
    # Check for docker-compose
    compose_path = shutil.which("docker-compose")
    if compose_path:
        print("✅ Docker Compose is available")
    else:
        print("⚠️ Docker Compose not found in PATH")
    
    # Check for project Docker files
    dockerfile = PROJECT_ROOT / "Dockerfile"
    composefile = PROJECT_ROOT / "docker-compose.yaml"
    
    print(f"\n📁 Docker Files:")
    print(f"   Dockerfile: {'✅ exists' if dockerfile.exists() else '❌ not found'}")
    print(f"   docker-compose.yaml: {'✅ exists' if composefile.exists() else '❌ not found'}")
    
else:
    print("❌ Docker is NOT installed")
    print("   Install from: https://docs.docker.com/get-docker/")

# Display Docker commands reference
print("\n📋 Docker Commands Reference:")
print("-" * 40)
print("""
# Build image
docker build -t noshow-api:latest .

# Run container
docker run -d --name noshow-api -p 8000:8000 \\
    -v $(pwd)/models/production:/app/models/production:ro \\
    noshow-api:latest

# View logs
docker logs -f noshow-api

# Stop and remove
docker stop noshow-api && docker rm noshow-api

# Using Docker Compose
docker-compose up -d        # Start
docker-compose logs -f      # View logs
docker-compose down         # Stop
""")

DOCKER DEPLOYMENT STATUS
✅ Docker is installed
   Docker version 28.4.0, build d8eb465
✅ Docker Compose is available

📁 Docker Files:
   Dockerfile: ✅ exists
   docker-compose.yaml: ✅ exists

📋 Docker Commands Reference:
----------------------------------------

# Build image
docker build -t noshow-api:latest .

# Run container
docker run -d --name noshow-api -p 8000:8000 \
    -v $(pwd)/models/production:/app/models/production:ro \
    noshow-api:latest

# View logs
docker logs -f noshow-api

# Stop and remove
docker stop noshow-api && docker rm noshow-api

# Using Docker Compose
docker-compose up -d        # Start
docker-compose logs -f      # View logs
docker-compose down         # Stop



In [ ]:
# =============================================================================
# CELL 26: API Documentation Overview
# =============================================================================

print("=" * 60)
print("API DOCUMENTATION")
print("=" * 60)

if check_api_status()["running"]:
    try:
        # Fetch OpenAPI specification
        response = requests.get(f"{API_URL}/openapi.json", timeout=5)
        
        if response.status_code == 200:
            openapi = response.json()
            
            print(f"\n📚 OpenAPI Specification:")
            print(f"   Title: {openapi['info']['title']}")
            print(f"   Version: {openapi['info']['version']}")
            print(f"   OpenAPI Version: {openapi.get('openapi', 'unknown')}")
            
            # Count endpoints by method
            methods = {}
            for path, path_info in openapi['paths'].items():
                for method in path_info.keys():
                    if method.upper() in ['GET', 'POST', 'PUT', 'DELETE', 'PATCH']:
                        methods[method.upper()] = methods.get(method.upper(), 0) + 1
            
            print(f"\n📋 Endpoints Summary:")
            print(f"   Total Paths: {len(openapi['paths'])}")
            for method, count in sorted(methods.items()):
                print(f"   {method}: {count} endpoints")
            
            # List all endpoints
            print(f"\n📑 All Endpoints:")
            for path, path_info in sorted(openapi['paths'].items()):
                for method, details in path_info.items():
                    if method.upper() in ['GET', 'POST', 'PUT', 'DELETE', 'PATCH']:
                        summary = details.get('summary', 'No description')
                        print(f"   {method.upper():6} {path}")
                        print(f"          └─ {summary}")
            
            print(f"\n🌐 Documentation URLs:")
            print(f"   Swagger UI: {API_URL}/docs")
            print(f"   ReDoc:      {API_URL}/redoc")
            print(f"   OpenAPI:    {API_URL}/openapi.json")
            
    except Exception as e:
        print(f"❌ Error fetching OpenAPI spec: {e}")
else:
    print("⚠️ API not running - cannot fetch documentation")
    print(f"\n   Start server, then visit:")
    print(f"   Swagger UI: {API_URL}/docs")
    print(f"   ReDoc:      {API_URL}/redoc")

API DOCUMENTATION

📚 OpenAPI Specification:
   Title: Healthcare No-Show Prediction API
   Version: 1.0.0
   OpenAPI Version: 3.1.0

📋 Endpoints Summary:
   Total Paths: 52
   DELETE: 2 endpoints
   GET: 28 endpoints
   POST: 22 endpoints

📑 All Endpoints:
   GET    /api/v1/auth/callback/oidc
          └─ Callback Oidc
   GET    /api/v1/auth/login/oidc
          └─ Login Oidc
   POST   /api/v1/auth/token
          └─ Login For Access Token
   GET    /api/v1/auth/users/me
          └─ Read Users Me
   GET    /api/v1/compliance/audit
          └─ View Audit Logs
   POST   /api/v1/compliance/consent
          └─ Record Consent
   POST   /api/v1/compliance/forget-patient
          └─ Forget Patient
   POST   /api/v1/feedback/
          └─ Submit Feedback
   GET    /api/v1/feedback/stats/{endpoint}
          └─ Get Feedback Stats
   GET    /api/v1/feedback/summary
          └─ Get Overall Feedback Summary
   GET    /api/v1/health
          └─ Health Check
   GET    /api/v1/live
          └─

In [ ]:
# =============================================================================
# CELL 27: Daily Operations Simulation
# =============================================================================

print("=" * 60)
print("SIMULATION: Daily Appointment Processing")
print("=" * 60)

if check_api_status()["running"]:
    # Generate realistic daily appointments
    np.random.seed(42)
    n_appointments = 100
    
    print(f"\n📅 Generating {n_appointments} appointments for today...")
    
    # Create realistic age distribution
    ages = np.concatenate([
        np.random.randint(5, 18, 10),     # Children
        np.random.randint(18, 35, 25),    # Young adults
        np.random.randint(35, 55, 35),    # Adults
        np.random.randint(55, 75, 20),    # Middle-aged
        np.random.randint(75, 90, 10),    # Elderly
    ])
    np.random.shuffle(ages)
    ages = ages[:n_appointments]
    
    # Generate appointments
    appointments = []
    for age in ages:
        appt = {
            "age": int(age),
            "gender": np.random.choice(["M", "F"], p=[0.4, 0.6]),
            "lead_days": max(0, int(np.random.exponential(10))),
            "sms_received": int(np.random.choice([0, 1], p=[0.15, 0.85])),
            "scholarship": int(np.random.choice([0, 1], p=[0.88, 0.12])),
            "hypertension": int(np.random.choice([0, 1], p=[0.75, 0.25]) if age > 40 else 0),
            "diabetes": int(np.random.choice([0, 1], p=[0.85, 0.15]) if age > 45 else 0),
        }
        appointments.append(appt)
    
    # Get predictions
    print("🔮 Getting predictions from API...")
    start_time = time.time()
    
    response = requests.post(
        f"{API_URL}/api/v1/predict/batch",
        json={"appointments": appointments}
    )
    
    elapsed_ms = (time.time() - start_time) * 1000
    
    if response.status_code == 200:
        result = response.json()
        summary = result["summary"]
        
        # Display operations summary
        print(f"\n{'─' * 50}")
        print("📊 DAILY OPERATIONS SUMMARY")
        print(f"{'─' * 50}")
        print(f"   Total Appointments: {summary['total']}")
        print(f"   Processing Time: {elapsed_ms:.0f}ms")
        print(f"{'─' * 50}")
        print(f"   Expected No-Shows: {summary['predicted_noshows']}")
        print(f"   Expected Attendance: {summary['predicted_shows']}")
        print(f"   No-Show Rate: {summary['predicted_noshows']/summary['total']:.1%}")
        print(f"   Average Risk Score: {summary['avg_probability']:.1%}")
        
        # Risk distribution
        print(f"\n{'─' * 50}")
        print("📈 RISK DISTRIBUTION")
        print(f"{'─' * 50}")
        
        risk_dist = summary['risk_distribution']
        for tier in ['CRITICAL', 'HIGH', 'MEDIUM', 'LOW', 'MINIMAL']:
            count = risk_dist.get(tier, 0)
            pct = count / summary['total'] * 100
            bar_length = int(pct / 2)
            bar = '█' * bar_length
            print(f"   {tier:10} │{bar:25}│ {count:3} ({pct:5.1f}%)")
        
        # Intervention plan
        critical = risk_dist.get('CRITICAL', 0)
        high = risk_dist.get('HIGH', 0)
        medium = risk_dist.get('MEDIUM', 0)
        
        print(f"\n{'─' * 50}")
        print("💡 INTERVENTION PLAN")
        print(f"{'─' * 50}")
        print(f"   📞 Phone calls needed:    {critical + high}")
        print(f"   📱 Extra SMS reminders:   {critical + high + medium}")
        print(f"   💰 Deposits to request:   {critical}")
        print(f"   ⭐ Priority scheduling:   {critical}")
        
        # Financial impact
        cost_per_noshow = 150
        intervention_success_rate = 0.30
        phone_call_cost = 5
        
        expected_noshows = summary['predicted_noshows']
        calls_to_make = critical + high
        prevented = int(calls_to_make * intervention_success_rate)
        savings = prevented * cost_per_noshow
        intervention_cost = calls_to_make * phone_call_cost
        net_savings = savings - intervention_cost
        
        print(f"\n{'─' * 50}")
        print("💰 FINANCIAL IMPACT ESTIMATE")
        print(f"{'─' * 50}")
        print(f"   Potential no-shows:       {expected_noshows}")
        print(f"   Potential loss:           ${expected_noshows * cost_per_noshow:,}")
        print(f"   Interventions planned:    {calls_to_make}")
        print(f"   Expected preventions:     {prevented}")
        print(f"   Intervention cost:        ${intervention_cost:,}")
        print(f"   Expected savings:         ${savings:,}")
        print(f"   Net savings:              ${net_savings:,}")
        print(f"{'─' * 50}")
        
    else:
        print(f"❌ Error: {response.text}")
else:
    print("⚠️ API not running - skipping simulation")

SIMULATION: Daily Appointment Processing

📅 Generating 100 appointments for today...
🔮 Getting predictions from API...

──────────────────────────────────────────────────
📊 DAILY OPERATIONS SUMMARY
──────────────────────────────────────────────────
   Total Appointments: 100
   Processing Time: 1381ms
──────────────────────────────────────────────────
   Expected No-Shows: 0
   Expected Attendance: 100
   No-Show Rate: 0.0%
   Average Risk Score: 26.1%

──────────────────────────────────────────────────
📈 RISK DISTRIBUTION
──────────────────────────────────────────────────
   CRITICAL   │                         │   0 (  0.0%)
   HIGH       │                         │   0 (  0.0%)
   MEDIUM     │█████████████████        │  35 ( 35.0%)
   LOW        │█████████████████████████████│  59 ( 59.0%)
   MINIMAL    │███                      │   6 (  6.0%)

──────────────────────────────────────────────────
💡 INTERVENTION PLAN
──────────────────────────────────────────────────
   📞 Phone calls n

In [ ]:
# =============================================================================
# CELL 28: Week 7 Summary
# =============================================================================

print("=" * 60)
print("WEEK 7 SUMMARY: MODEL DEPLOYMENT")
print("=" * 60)

summary = """
✅ COMPLETED TASKS:
   1. Prepared production model with metadata
   2. Explored API architecture and schemas
   3. Tested all API endpoints
   4. Benchmarked API performance
   5. Demonstrated Python client usage
   6. Reviewed Docker deployment options
   7. Simulated real-world integration

📁 PRODUCTION FILES CREATED:
   models/production/
   ├── model.joblib           # Production model
   ├── preprocessor.joblib    # Preprocessor (if available)
   └── model_metadata.json    # Model metadata

🌐 API ENDPOINTS:
   Health & Monitoring:
   ├── GET  /health              → Health check
   ├── GET  /ready               → Readiness probe
   └── GET  /live                → Liveness probe
   
   Predictions:
   ├── POST /api/v1/predict      → Single prediction
   ├── POST /api/v1/predict/batch → Batch predictions
   ├── POST /api/v1/predict/quick → Quick prediction
   └── GET  /api/v1/predict/thresholds → Threshold info
   
   Model Info:
   ├── GET  /api/v1/model        → Model information
   ├── GET  /api/v1/model/features → Feature info
   └── POST /api/v1/model/reload → Reload model

🐳 DEPLOYMENT OPTIONS:
   • Direct:     python serve_model.py
   • Docker:     docker-compose up -d
   • Production: docker-compose -f ... -f docker-compose.prod.yaml up

📊 TYPICAL PERFORMANCE:
   • Single prediction:    10-50ms
   • Batch (100 items):    100-200ms
   • Concurrent requests:  50-100 req/sec
"""
print(summary)

WEEK 7 SUMMARY: MODEL DEPLOYMENT

✅ COMPLETED TASKS:
   1. Prepared production model with metadata
   2. Explored API architecture and schemas
   3. Tested all API endpoints
   4. Benchmarked API performance
   5. Demonstrated Python client usage
   6. Reviewed Docker deployment options
   7. Simulated real-world integration

📁 PRODUCTION FILES CREATED:
   models/production/
   ├── model.joblib           # Production model
   ├── preprocessor.joblib    # Preprocessor (if available)
   └── model_metadata.json    # Model metadata

🌐 API ENDPOINTS:
   Health & Monitoring:
   ├── GET  /health              → Health check
   ├── GET  /ready               → Readiness probe
   └── GET  /live                → Liveness probe

   Predictions:
   ├── POST /api/v1/predict      → Single prediction
   ├── POST /api/v1/predict/batch → Batch predictions
   ├── POST /api/v1/predict/quick → Quick prediction
   └── GET  /api/v1/predict/thresholds → Threshold info

   Model Info:
   ├── GET  /api/v1/model 

In [ ]:
# =============================================================================
# CELL 29: Month 2 Completion
# =============================================================================

print("\n" + "🎉 " * 20)
print("\n" + " " * 20 + "MONTH 2 COMPLETE!")
print(" " * 10 + "Applied ML & MLOps Lite - FINISHED")
print("\n" + "🎉 " * 20)

completion_message = """

Congratulations! You have successfully completed Month 2.

📚 WHAT YOU'VE LEARNED:

   Week 5: Supervised Learning 1
   ├── ML problem framing
   ├── Train/test splitting
   ├── Baseline model training
   └── Model evaluation metrics

   Week 6: Supervised Learning 2
   ├── sklearn pipelines
   ├── Hyperparameter tuning
   ├── SHAP interpretability
   └── Threshold optimization

   Week 7: MLOps Lite
   ├── FastAPI development
   ├── REST API design
   ├── Docker containerization
   └── API testing & documentation

🚀 YOUR COMPLETE SYSTEM:

   Data → Pipeline → Training → Tuning → API → Deployment
            ↑                                      ↓
        (Month 1)                           (Production)

🎯 NEXT STEPS:
   • Deploy to cloud (AWS/GCP/Azure)
   • Add authentication & authorization
   • Set up CI/CD pipeline
   • Implement monitoring & alerting
   • Add A/B testing framework

You are now ready for real-world ML projects!

"""
print(completion_message)


🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 

                    MONTH 2 COMPLETE!
          Applied ML & MLOps Lite - FINISHED

🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 


Congratulations! You have successfully completed Month 2.

📚 WHAT YOU'VE LEARNED:

   Week 5: Supervised Learning 1
   ├── ML problem framing
   ├── Train/test splitting
   ├── Baseline model training
   └── Model evaluation metrics

   Week 6: Supervised Learning 2
   ├── sklearn pipelines
   ├── Hyperparameter tuning
   ├── SHAP interpretability
   └── Threshold optimization

   Week 7: MLOps Lite
   ├── FastAPI development
   ├── REST API design
   ├── Docker containerization
   └── API testing & documentation

🚀 YOUR COMPLETE SYSTEM:

   Data → Pipeline → Training → Tuning → API → Deployment
            ↑                                      ↓
        (Month 1)                           (Production)

🎯 NEXT STEPS:
   • Deploy to cloud (AWS/GCP/Azure)
   • Add authentication & authorization
   • Set up CI/CD pipelin

In [ ]:
# =============================================================================
# CELL 30: Cleanup Instructions
# =============================================================================

print("=" * 60)
print("CLEANUP INSTRUCTIONS")
print("=" * 60)

cleanup_text = """
If you started the API server, remember to stop it:

OPTION 1: Terminal
   Press Ctrl+C in the terminal running serve_model.py

OPTION 2: Docker
   docker-compose down

OPTION 3: Find and Kill Process
   
   Linux/Mac:
   $ lsof -i :8000 | grep python | awk '{print $2}' | xargs kill
   
   Windows (PowerShell):
   > Get-Process -Id (Get-NetTCPConnection -LocalPort 8000).OwningProcess | Stop-Process

CLEANUP COMMANDS:

   # Remove Docker containers and images
   docker-compose down --rmi all
   
   # Clean Python cache
   find . -type d -name __pycache__ -exec rm -rf {} +
   find . -type f -name "*.pyc" -delete
   
   # Clean output directories (optional)
   rm -rf outputs/experiments/*
   rm -rf logs/*
"""
print(cleanup_text)

print("\n" + "=" * 60)
print("END OF WEEK 7 NOTEBOOK")
print("=" * 60)

CLEANUP INSTRUCTIONS

If you started the API server, remember to stop it:

OPTION 1: Terminal
   Press Ctrl+C in the terminal running serve_model.py

OPTION 2: Docker
   docker-compose down

OPTION 3: Find and Kill Process

   Linux/Mac:
   $ lsof -i :8000 | grep python | awk '{print $2}' | xargs kill

   Windows (PowerShell):
   > Get-Process -Id (Get-NetTCPConnection -LocalPort 8000).OwningProcess | Stop-Process

CLEANUP COMMANDS:

   # Remove Docker containers and images
   docker-compose down --rmi all

   # Clean Python cache
   find . -type d -name __pycache__ -exec rm -rf {} +
   find . -type f -name "*.pyc" -delete

   # Clean output directories (optional)
   rm -rf outputs/experiments/*
   rm -rf logs/*


END OF WEEK 7 NOTEBOOK
